In [2]:
import re
import pandas as pd
from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline
)

In [3]:
# Load data
df = pd.read_csv("model_subset_500.csv")
text_column = "no_stopwords_post_content"
df = df.dropna(subset=[text_column])
texts = df[text_column].tolist()

In [ ]:
# FLAN-T5 CONDITION PREDICTION 
cond_tok   = AutoTokenizer.from_pretrained("google/flan-t5-base")
cond_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

def predict_condition(text):
    prompt = (
        "What mental health condition and severity is described in the following post?\n\n"
        f"Post: {text}\n\n"
        "Answer:"
    )
    inp = cond_tok(prompt, return_tensors="pt").input_ids
    out = cond_model.generate(inp, max_length=30)
    result = cond_tok.decode(out[0], skip_special_tokens=True).strip()

    # parsing logic 
    if "severity" in result.lower():
        parts = re.split(r"Severity:", result, flags=re.IGNORECASE)
        condition = parts[0].strip()
    elif ":" in result:
        condition = result.split(":", 1)[0].strip()
    else:
        condition = result

    return condition.lower()

In [5]:
# ZERO-SHOT SEVERITY PREDICTION
zs = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=0  
)

Device set to use mps:0


In [ ]:
def predict_severity(text):
    labels = ["mild", "moderate", "critical"]
    hypothesis_template = "The severity of this post is {}."
    res = zs(
        text,
        labels,
        hypothesis_template=hypothesis_template,
        multi_label=False
    )
    return res["labels"][0].lower()

In [8]:
# Run both
conds = []
sevs  = []
for post in tqdm(texts, desc="Classifying posts"):
    conds.append(predict_condition(post))
    sevs .append(predict_severity(post))

# save results
df["predicted_condition"] = conds
df["predicted_severity"]  = sevs
df.to_csv("flan_zs.csv", index=False)
print("Saved as flan_zs.csv")

Classifying posts: 100%|██████████| 500/500 [05:37<00:00,  1.48it/s]


Saved as flan_zs.csv


In [10]:
df.head(10)

,forum_name,thread_id,post_timestamp,author,post_content_llm,no_of_comments,comment_authors,comments_content_llm,authors_replies_llm,comment_list,no_stopwords_post_content,no_stopwords_comments_content,no_stopwords_author_comment,predicted_condition,predicted_severity
0,anxiety,359261,2017-11-27 14:29:00,lmslms,hi everyone. first time user here. i have depr...,5,"['JessF', 'startingnew', 'startingnew', 'Ashle...","hello lmslms, welcome here. in answers to your...",yes i touch with my gp and all. yes i am the c...,"[""JessF: Hello lmslms, welcome here. In answe...",hi everyone first time user depression panic d...,hello lmslms welcome answers last question thi...,yes touch gp yes classic dr google know help a...,depression,critical
1,depression,369653,2018-04-21 05:21:00,Ddom,finally got over my anxiety for now lol to ask...,3,"['PamelaR', 'PamelaR', 'white knight']","hello ddom wow, you have had a life of difficu...","hello pamela, thank you for you for taking the...","[""PamelaR: Hello Ddom Wow, you've had a life o...",finally got anxiety lol ask advice strategies ...,hello ddom wow life difficulties hey good foun...,hello pamela thank taking time respond answer ...,depression,moderate
2,ptsd-trauma,31314,2021-07-15 13:38:00,DisplayName5742,"hello fellow members, writing here today as i ...",7,"['jtjt_4862', 'quirkywords', 'topsy_', 'jtjt_4...","hi displayname5742, thank you for sharing your...","hello jt, thank you for the response. that doe...","['jtjt_4862: Hi DisplayName5742, Thank you for...",hello fellow members writing today feel unable...,hi displayname5742 thank sharing story sorry h...,hello jt thank response makes sense forgiving ...,teen,moderate
3,anxiety,557173,2023-01-18 19:59:00,Heidi_1,"hey everyone, this year has not gone the way i...",2,"['smallwolf', 'therising']",hello. when things do not seem to be getting b...,NaN,"[""smallwolf: hello. when things don't seem to ...",hey everyone year gone way want start looks li...,hello things seem getting better natural wonde...,NaN,apathy,critical
4,depression,119125,2015-09-12 12:33:00,Robbie81,"hi, i have recently been diagnosed with depres...",3,"['white knight', 'white knight', 'white knight']","white knight: hi cammo, i am not in the ""spher...","hi tony wk, thanks for your reply. i have been...","['white knight: Hi Cammo, I am not in the ""sph...",hi recently diagnosed depression anxiety belie...,white knight hi cammo sphere reply ask words m...,hi tony wk thanks reply contact birth mother s...,depression,critical
5,depression,581763,2024-01-03 14:04:00,nurse91,this is going to be a long one. partner and i ...,4,"['Sophie_M', 'therising', 'yggdrasil', 'Sereni...","hi nurse91, welcome to the forums and thank yo...",NaN,"[""Sophie_M: Hi nurse91, Welcome to the forum...",going long one partner togther nearly 7 years ...,hi nurse91 welcome forums thank bravery share ...,NaN,depression,critical
6,anxiety,484115,2019-05-28 21:22:00,One_step_closer_to_recove,"hi all, i am currently going through a medical...",1,['Croix'],dear one step~ the constant noise in your ears...,NaN,"[""Croix: Dear One Step~ The constant noise in ...",hi currently going medical issues causing live...,dear one step constant noise ears lack balance...,NaN,anxiety,critical
7,depression,513148,2020-09-18 10:55:00,Leigh1987,"hi all, i went to the gp again last night look...",3,"['golden82', 'Ely_', 'No one on earth']","hi leigh, my understanding is to 'fit' the dia...","thanks guys, i looked at the 9 criteria and do...","[""golden82: hi Leigh, My understanding is to '...",hi went gp last night looking relief constant ...,hi leigh understanding fit diagnosis bpd need ...,thanks guys looked 9 criteria really meet gues...,depression,moderate
8,anxiety,231407,2016-06-21 13:45:00,Dwwmills,hello everyone. i am just wondering whether an...,4,"['White_Rose', 'pipsy', 'White_Rose', 'white k...",hello dean i have had a couple of sleep studie...,thanks for the replies mary and lynda. i am no...,['White_Rose: 